# Hoja de trabajo 2

Link del repositorio: https://github.com/faguilarleal/HDT2_deep  


Integrantes:  
- Franci Aguilar 22243
- César López 22404


## Ejercicio 1. Experimentación práctica
#### Task 1 - Preparación del conjunto de datos
Cargue el conjunto de datos de Iris utilizando bibliotecas como sklearn.datasets. Luego, divida el conjunto de datos en conjuntos de entrenamiento y validación.


In [4]:
import numpy as np
import copy
import matplotlib.pyplot as plt
import scipy
from PIL import Image
import os
from collections import defaultdict
import torch
from torch import nn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd

In [5]:
# Cargar el conjunto de datos Iris
iris = load_iris()

# Convertir a DataFrame para verlo más ordenado
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target

print("Primeras filas del dataset:")
print(df.head())

X = iris.data
y = iris.target

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("\nTamaños de los conjuntos:")
print("Entrenamiento:", X_train.shape, y_train.shape)
print("Validación:", X_val.shape, y_val.shape)


Primeras filas del dataset:
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target  
0       0  
1       0  
2       0  
3       0  
4       0  

Tamaños de los conjuntos:
Entrenamiento: (120, 4) (120,)
Validación: (30, 4) (30,)


#### Task 2 - Arquitectura modelo
Cree una red neuronal feedforward simple utilizando nn.Module de PyTorch. Luego, defina capa de entrada, capas ocultas y capa de salida. Después, elija las funciones de activación y el número de neuronas por capa.

In [6]:
import torch.nn.functional as F

class IrisNet(nn.Module):
    def __init__(self):
        super(IrisNet, self).__init__()
        # Capa de entrada -> 4 neuronas 
        # Primera capa oculta -> 16 neuronas
        # Segunda capa oculta -> 8 neuronas
        # Capa de salida -> 3 neuronas (porque iris tiene 3 clases)
        self.fc1 = nn.Linear(4, 16)   # entrada -> capa oculta 1
        self.fc2 = nn.Linear(16, 8)   # capa oculta 1 -> capa oculta 2
        self.fc3 = nn.Linear(8, 3)    # capa oculta 2 -> salida

    def forward(self, x):
        # Funciones de activación
        x = F.relu(self.fc1(x))  
        x = F.relu(self.fc2(x))  
        x = self.fc3(x)         
        return x

# Instanciar el modelo
model = IrisNet()
print(model)


IrisNet(
  (fc1): Linear(in_features=4, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=3, bias=True)
)


#### Task 3 - Funciones de Pérdida
Utilice diferentes funciones de pérdida comunes como Cross-Entropy Loss y MSE para clasificación. Entrene el modelo con diferentes funciones de pérdida y registre las pérdidas de entrenamiento y test. Debe utilizar al menos 3 diferentes funciones. Es decir, procure que su código sea capaz de parametrizar el uso de diferentes funciones de pérdida.

In [7]:
import torch.optim as optim

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_val   = torch.tensor(X_val, dtype=torch.float32)
y_val   = torch.tensor(y_val, dtype=torch.long)

#  Funciones de pérdida disponibles
loss_functions = {
    "CrossEntropy": nn.CrossEntropyLoss(),
    "MSE": nn.MSELoss(),  # MSE requiere one-hot en y
    "NLLLoss": nn.NLLLoss()  # requiere log_softmax en salida
}


#  Entrenamiento parametrizable
def train_model(loss_name, epochs=50, lr=0.01):
    model = IrisNet()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    criterion = loss_functions[loss_name]

    # Para NLL necesitamos modificar el forward (añadir log_softmax)
    use_log_softmax = (loss_name == "NLLLoss")

    # Para MSE necesitamos one-hot encoding de las etiquetas
    if loss_name == "MSE":
        y_train_oh = torch.nn.functional.one_hot(y_train, num_classes=3).float()
        y_val_oh   = torch.nn.functional.one_hot(y_val, num_classes=3).float()
    else:
        y_train_oh, y_val_oh = y_train, y_val

    history = {"train_loss": [], "val_loss": []}

    for epoch in range(epochs):
        # --- Forward ---
        outputs = model(X_train)
        if use_log_softmax:
            outputs = torch.log_softmax(outputs, dim=1)

        loss = criterion(outputs, y_train_oh)

        # --- Backprop ---
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # --- Evaluación en validación ---
        with torch.no_grad():
            val_outputs = model(X_val)
            if use_log_softmax:
                val_outputs = torch.log_softmax(val_outputs, dim=1)
            val_loss = criterion(val_outputs, y_val_oh)

        history["train_loss"].append(loss.item())
        history["val_loss"].append(val_loss.item())

        if (epoch+1) % 10 == 0:
            print(f"[{loss_name}] Epoch {epoch+1}/{epochs} "
                  f"Train Loss: {loss.item():.4f} | Val Loss: {val_loss.item():.4f}")

    return history

# -------------------------------
# 5. Entrenar con las tres funciones
# -------------------------------
hist_ce  = train_model("CrossEntropy", epochs=50)
hist_mse = train_model("MSE", epochs=50)
hist_nll = train_model("NLLLoss", epochs=50)

[CrossEntropy] Epoch 10/50 Train Loss: 0.9309 | Val Loss: 0.9104
[CrossEntropy] Epoch 20/50 Train Loss: 0.6141 | Val Loss: 0.5871
[CrossEntropy] Epoch 30/50 Train Loss: 0.3787 | Val Loss: 0.3689
[CrossEntropy] Epoch 40/50 Train Loss: 0.2358 | Val Loss: 0.2347
[CrossEntropy] Epoch 50/50 Train Loss: 0.1307 | Val Loss: 0.1371
[MSE] Epoch 10/50 Train Loss: 0.3377 | Val Loss: 0.3335
[MSE] Epoch 20/50 Train Loss: 0.2429 | Val Loss: 0.2261
[MSE] Epoch 30/50 Train Loss: 0.1848 | Val Loss: 0.1767
[MSE] Epoch 40/50 Train Loss: 0.1370 | Val Loss: 0.1334
[MSE] Epoch 50/50 Train Loss: 0.1176 | Val Loss: 0.1178
[NLLLoss] Epoch 10/50 Train Loss: 0.8780 | Val Loss: 0.8398
[NLLLoss] Epoch 20/50 Train Loss: 0.5714 | Val Loss: 0.5461
[NLLLoss] Epoch 30/50 Train Loss: 0.3606 | Val Loss: 0.3477
[NLLLoss] Epoch 40/50 Train Loss: 0.2195 | Val Loss: 0.2180
[NLLLoss] Epoch 50/50 Train Loss: 0.1300 | Val Loss: 0.1392


#### Task 4 - Técnicas de Regularización

Utilice distintas técnicas de regularización como L1, L2 y dropout. Entrene el modelo con y sin técnicas de
regularización y observe el impacto en el overfitting y la generalización. Debe utilizar al menos 3 diferentes técnicas.
Es decir, procure que su código sea capaz de parametrizar el uso de diferentes técnicas de regularización

#### Task 5 - Algoritmos de Optimización

Utilice distintas técnicas de optimización como SGD, Batch GD, Mini-Batch GD. Entrene el modelo con algoritmos de
optimización y registre las pérdidas y tiempos de entrenamiento y test. Debe utilizar al menos 3 diferentes algoritmos.
Es decir, procure que su código sea capaz de parametrizar el uso de diferentes algoritmos de optimización

#### Task 6 - Experimentación y Análisis

Entrene los modelos con diferentes combinaciones de funciones de pérdida, técnicas de regularización y algoritmos
de optimización. Para no complicar esta parte, puede dejar fijo dos de estos parámetros (función de pérdida, técnicas de regularización, algoritmo de optimización) y solamente cambiar uno de ellos. Deben verse al menos 9
combinaciones en total, donde es válido que en una de ellas no haya ninguna técnica de regularización. Si quiere
experimentar con más combinaciones se le dará hasta 10% de puntos extra.
Para cada combinación registre métricas como precisión, pérdida y alguna otra métrica que considere pertinente
(Recuerde lo visto en inteligencia artificial).
Visualice las curvas (tanto en precisión, pérdida y la tercera métrica que decidió) de entrenamiento y validación
utilizando bibliotecas como matplotlib y/o seaborn. Además, recuerde llevar tracking de los tiempos de ejecución de
cada combinación.


#### Task 7- Discusión


Discuta los resultados obtenidos de diferentes modelos. Compare la velocidad de convergencia y el rendimiento final
de modelos utilizando diferentes funciones de pérdida, técnicas de regularización, y algoritmos de optimización.
Explore y discuta por qué ciertas técnicas podrían conducir a un mejor rendimiento. tanto técnicas de regularización,
funciones de pérdida como algoritmos de optimización.

## Ejercicio 2. Repaso de teoría

1. ¿Cuál es la principal innovación de la arquitectura Transformer?

    La gran idea del Transformer es que deja de lado las redes recurrentes y las convoluciones y, en su lugar, usa atención para que cada palabra pueda mirar a las demás de una sola vez; así el modelo entiende relaciones lejanas más fácil, entrena en paralelo (más rápido) y solo necesita agregar una pista del orden de las palabras con “positional encodings”.

2. ¿Cómo funciona el mecanismo de atención del scaled dot-product?

    Cada palabra se compara con todas las otras para ver a cuáles debería “prestarles más atención”; esas comparaciones se convierten en pesos (que suman 1) y se usan para mezclar la información de las palabras importantes. El “escalado” solo evita que los números exploten y el softmax dé resultados raros y en el decodificador se tapa lo que viene después para no hacer trampa.

3. ¿Por qué se utiliza la atención de múltiples cabezales en Transformer?

    Se usan varias “cabezas” de atención porque cada una puede fijarse en cosas distintas al mismo tiempo: una puede mirar concordancias cercanas, otra dependencias lejanas, otra nombres propios, etc. Al juntar todas, el modelo capta más patrones y entiende mejor el texto que si tuviera una sola mirada.

4. ¿Cómo se incorporan los positional encodings en el modelo Transformer?

    Como la atención por sí sola no sabe el orden, al embedding de cada palabra se le suma un vector que depende de su posición los positional encodings. Es como ponerle a cada palabra un GPS de “estoy en la posición 1, 2, 3…”, para que el modelo distinga “el perro muerde al gato” de “el gato muerde al perro”.

5. ¿Cuáles son algunas aplicaciones de la arquitectura Transformer más allá de la machine translation?

    Además de traducir, los Transformers se usan para un montón de cosas: escribir o resumir textos, responder preguntas y chatear, entender y generar código, analizar imágenes, trabajar con audio y voz, combinar texto-imagen, e incluso para datos como secuencias biológicas o series de tiempo. Son versátiles porque la misma idea de atención se adapta a muchos tipos de datos.